In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# test suite for filter option
import copy
import json
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
import xarray as xr

import semantique as sq
from semantique.processor.core import QueryProcessor, FilterProcessor

In [3]:
# data cube connection with specific timestamps
import pystac
from pystac_client import Client
from shapely.geometry import box

# define temporal & spatial range to perform STAC query
xmin, ymin, xmax, ymax = -2.75, 47.25, -2.25, 47.75
aoi = box(xmin, ymin, xmax, ymax)
t_range = ["2019-07-15", "2020-08-01"]

# STAC-based metadata retrieval
catalog = Client.open("https://earth-search.aws.element84.com/v1")
query = catalog.search(
    collections="sentinel-2-l2a", 
    datetime=t_range, 
    limit=100, 
    intersects=aoi
)
stac_json = query.item_collection_as_dict()
gdf = gpd.GeoDataFrame.from_features(stac_json, "epsg:4326")

In [4]:
class TestSuite:
    def __init__(self):
        # create an empty list to store test cases
        self.tests = []
        # define the maximum number of items in the result
        self.max_items = None
        # define general context config
        with open("../files/mapping.json", "r") as file:
            mapping = sq.mapping.Semantique(json.load(file))
        with open("../files/layout_gtiff.json", "r") as file:
            dc = sq.datacube.GeotiffArchive(json.load(file), src = "../files/layers_gtiff.zip")
        space = sq.SpatialExtent(gpd.read_file("../files/footprint.geojson"))
        time = sq.TemporalExtent("2019-01-01", "2020-12-31")
        self.context = {
            "datacube": dc, 
            "mapping": mapping,
            "space": space,
            "time": time,
            "crs": 3035, 
            "tz": "UTC", 
            "spatial_resolution": [-10, 10],
            "track_types": False,
            # "meta_timestamps": gdf.datetime.drop_duplicates()
        }
        
    def populate(self):
        # define base entities
        recipe = sq.QueryRecipe()
        red_band = sq.reflectance("s2_band04")
        green_band = sq.reflectance("s2_band03")
        blue_band = sq.reflectance("s2_band02")

        # define test no. 1
        recipe = sq.QueryRecipe()
        recipe["composite"] = sq.collection(red_band, green_band, blue_band).\
            concatenate("band").\
            filter(sq.entity("cloud").evaluate("not"))
        result = {
            'atmosphere_colortype': self.max_items,
            'reflectance_s2_band02': self.max_items,
            'reflectance_s2_band03': self.max_items,
            'reflectance_s2_band04': self.max_items
        }
        self.tests.append({
            "name": "no.1",
            "desc": "No temporal filter",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 2a 
        # test recipe
        recipe = sq.QueryRecipe()
        recipe["composite"] = sq.collection(
                blue_band.filter(sq.self().extract("time").evaluate("during", sq.time_interval("2021-01-01", "2021-12-31"))),
                green_band.filter_time("year", "greater", 2020).filter_time("year", "less", 2022), 
                red_band.filter(sq.self().extract("time", "year").evaluate("less", 2015)) 
            ).\
            concatenate("band")
        # expected number of items in result
        result = {
            'reflectance_s2_band02': 8737,
            'reflectance_s2_band03': 8760,
            'reflectance_s2_band04': 482136
        }
        # test & expected result + test description
        self.tests.append({
            "name": "no.2a",
            "desc": "Various temporal filters applied directly to data layers",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 2b
        recipe = sq.QueryRecipe()
        recipe["blue_I"] = sq.collection(
                blue_band.filter(sq.self().extract("time").evaluate("during", sq.time_interval("2021-01-01", "2021-12-31"))),
                blue_band.filter(sq.self().extract("time").evaluate("before", sq.time_instant("2021-01-01"))),
            ).\
            concatenate("band")
        recipe["blue_II"] = sq.collection(
                blue_band.filter(sq.self().extract("time").evaluate("after", sq.time_instant("2021-12-31")))
            ).\
            concatenate("band")
        recipe["red_green"] = sq.collection(
                green_band.filter_time("year", "less", 2015),
                red_band.filter_time("year", "greater", 2010).filter_time("year", "less", 2020)
            ).\
            concatenate("band")
        result = {
            'reflectance_s2_band02': self.max_items,
            'reflectance_s2_band03': 482136,
            'reflectance_s2_band04': 78888
        }
        self.tests.append({
            "name": "no.2b",
            "desc": "Temporal filters across different results",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 3a
        recipe = sq.QueryRecipe()
        recipe["composite"] = sq.collection(red_band, green_band, blue_band).\
            concatenate("band").\
            filter(sq.entity("cloud").evaluate("not")).\
            evaluate("subtract", blue_band.filter_time("before", sq.time_instant("2019-12-31")))
        result = {
            'atmosphere_colortype': 525936,
            'reflectance_s2_band02': 525936,
            'reflectance_s2_band03': 525936,
            'reflectance_s2_band04': 525936
        }
        self.tests.append({
            "name": "no.3a",
            "desc": "Temporal filter applied indirectly via evaluate as part of filter object (algebraic operators)",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 3b
        recipe = sq.QueryRecipe()
        recipe["composite"] = sq.collection(red_band, green_band).\
            concatenate("band").\
            filter(sq.entity("cloud").evaluate("not")).\
            evaluate("or", blue_band.filter_time("before", sq.time_instant("2019-12-31")))
        result = {
            'atmosphere_colortype': self.max_items,
            'reflectance_s2_band02': 525936,
            'reflectance_s2_band03': self.max_items,
            'reflectance_s2_band04': self.max_items
        }
        self.tests.append({
            "name": "no.3b",
            "desc": "Temporal filter applied indirectly via evaluate as part of filter object (boolean/relational/membership operators)",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })
      
        # define test no. 3c
        recipe = sq.QueryRecipe()
        recipe["composite"] = sq.collection(red_band, green_band, blue_band).\
            concatenate("band").\
            filter(sq.entity("cloud").evaluate("not")).\
            filter_time("before", sq.time_instant("2019-12-31")).\
            evaluate("subtract", blue_band)
        result = {
            'atmosphere_colortype': 525936,
            'reflectance_s2_band02': 525936,
            'reflectance_s2_band03': 525936,
            'reflectance_s2_band04': 525936
        }
        self.tests.append({
            "name": "no.3c",
            "desc": "Temporal filter applied indirectly via evaluate as part of object to be filtered (algebraic operators), output equivalent to 3a",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 3d
        recipe = sq.QueryRecipe()
        recipe["composite"] = sq.collection(red_band, green_band).\
            concatenate("band").\
            filter(sq.entity("cloud").evaluate("not")).\
            filter_time("before", sq.time_instant("2019-12-31")).\
            evaluate("or", blue_band)
        result = {
            'atmosphere_colortype': 525936,
            'reflectance_s2_band02': self.max_items,
            'reflectance_s2_band03': 525936,
            'reflectance_s2_band04': 525936
        }
        self.tests.append({
            "name": "no.3d",
            "desc": "Temporal filter applied indirectly via evaluate as part of object to be filtered (boolean/relational/membership operators), output should be equivalent to 3b but is not due to the way NaNs are handled (https://github.com/ZGIS/semantique/issues/54)",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 3e
        recipe = sq.QueryRecipe()
        recipe["composite"] = sq.collection(green_band).\
            concatenate("band").\
            filter(sq.entity("cloud").filter_time("before", sq.time_instant("2019-12-31")).evaluate("not")).\
            evaluate("subtract", blue_band)
        result = {
            'atmosphere_colortype': 525936,
            'reflectance_s2_band02': 525936,
            'reflectance_s2_band03': 525936
        }
        self.tests.append({
            "name": "no.3e",
            "desc": "Temporal filter applied indirectly via filter with entity",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 3f
        recipe = sq.QueryRecipe()
        recipe["composite"] = sq.collection(green_band, blue_band).\
            concatenate("band").\
            filter(sq.entity("cloud").filter_time("before", sq.time_instant("2019-12-31")).evaluate("not")).\
            evaluate("subtract", blue_band)
        recipe["composite_II"] = sq.collection(red_band, blue_band).\
            concatenate("band").\
            filter(sq.entity("cloud").evaluate("not")).\
            evaluate("or", blue_band.filter_time("before", sq.time_instant("2019-12-31")))
        result = {
            'atmosphere_colortype': self.max_items,
            'reflectance_s2_band02': self.max_items,
            'reflectance_s2_band03': 525936,
            'reflectance_s2_band04': self.max_items
        }
        self.tests.append({
            "name": "no.3f",
            "desc": "Temporal filter applied indirectly via filter with entity",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 4a
        recipe = sq.QueryRecipe()
        recipe["composite"] = sq.collection(red_band, green_band, blue_band).\
            concatenate("band").\
            filter(
                sq.collection(
                    sq.self().extract("time", "year").evaluate("greater", 2015),
                    sq.self().extract("time", "year").evaluate("less", 2020),
                    ).merge("all")
            ).\
            filter(sq.entity("cloud").evaluate("not")).\
            evaluate("subtract", blue_band)
        result = {
            'atmosphere_colortype': 35064,
            'reflectance_s2_band02': 35064,
            'reflectance_s2_band03': 35064,
            'reflectance_s2_band04': 35064
        }
        self.tests.append({
            "name": "no.4a",
            "desc": "multiple temporal filter organised in a collection",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 4b
        recipe = sq.QueryRecipe()
        recipe["composite"] = sq.collection(red_band, green_band, blue_band).\
            concatenate("band").\
            filter(
                sq.collection(
                    sq.self().extract("time", "year").evaluate("greater", 2015),
                    sq.self().extract("time", "year").evaluate("less", 2020),
                    sq.self().evaluate("less", 1)
                    ).merge("all")
            ).\
            filter(sq.entity("cloud").evaluate("not")).\
            evaluate("subtract", blue_band)
        result = {
            'atmosphere_colortype': 35064,
            'reflectance_s2_band02': 35064,
            'reflectance_s2_band03': 35064,
            'reflectance_s2_band04': 35064
        }
        self.tests.append({
            "name": "no.4b",
            "desc": "multiple temporal filter and non-blocking non-temporal filter organised in a collection",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 4c
        recipe = sq.QueryRecipe()
        recipe["composite"] = sq.collection(red_band, green_band, blue_band).\
            concatenate("band").\
            filter(
                sq.collection(
                    sq.self().extract("time", "year").evaluate("less", 2000),
                    sq.self().extract("time", "year").evaluate("greater", 2020),
                    sq.self().evaluate("less", 1)
                    ).merge("any")
            ).\
            filter(sq.entity("cloud").evaluate("not")).\
            evaluate("subtract", blue_band)
        result = {
            'atmosphere_colortype': self.max_items,
            'reflectance_s2_band02': self.max_items,
            'reflectance_s2_band03': self.max_items,
            'reflectance_s2_band04': self.max_items
        }
        self.tests.append({
            "name": "no.4c",
            "desc": "multiple temporal filter and blocking non-temporal filter organised in a collection",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 5a
        recipe = sq.QueryRecipe()
        recipe["composite"] = sq.collection(red_band, green_band, blue_band).\
            concatenate("band").\
            filter_time("before", sq.time_instant("2019-12-31")).\
            fill("time", "nearest")
        result = {
            'reflectance_s2_band02': 525936,
            'reflectance_s2_band03': 525936,
            'reflectance_s2_band04': 525936
        }
        self.tests.append({
            "name": "no.5a",
            "desc": "temporal filter followed by fill operation",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 5b
        recipe = sq.QueryRecipe()
        recipe["composite"] = sq.collection(red_band, green_band, blue_band).\
            concatenate("band").\
            fill("time", "nearest").\
            filter_time("before", sq.time_instant("2019-12-31"))
        result = {
            'reflectance_s2_band02': self.max_items,
            'reflectance_s2_band03': self.max_items,
            'reflectance_s2_band04': self.max_items,
        }
        self.tests.append({
            "name": "no.5b",
            "desc": "fill operation followed by temporal filter",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 5c
        recipe = sq.QueryRecipe()
        recipe["composite"] = sq.collection(red_band, green_band, blue_band).\
            concatenate("band").\
            filter_time("before", sq.time_instant("2019-12-31")).\
            smooth("mean", "time", 3)
        result = {
            'reflectance_s2_band02': 525936,
            'reflectance_s2_band03': 525936,
            'reflectance_s2_band04': 525936
        }
        self.tests.append({
            "name": "no.5c",
            "desc": "temporal filter followed by smooth operation",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 5d
        recipe = sq.QueryRecipe()
        recipe["composite"] = sq.collection(red_band, green_band, blue_band).\
            concatenate("band").\
            smooth("mean", "time", 3).\
            filter_time("before", sq.time_instant("2019-12-31"))
        result = {
            'reflectance_s2_band02': self.max_items,
            'reflectance_s2_band03': self.max_items,
            'reflectance_s2_band04': self.max_items,
        }
        self.tests.append({
            "name": "no.5d",
            "desc": "smooth operation followed by temporal filter",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 6a
        recipe = sq.QueryRecipe()
        recipe["composite"] = sq.collection(red_band, green_band, blue_band).\
            concatenate("band").\
            filter_time("before", sq.time_instant("2019-12-31")).\
            assign(0, at = sq.self().evaluate("less", 2))
        result = {
            'reflectance_s2_band02': 525936,
            'reflectance_s2_band03': 525936,
            'reflectance_s2_band04': 525936,
        }
        self.tests.append({
            "name": "no.6a",
            "desc": "temporal filter followed by assign operation",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 6b
        recipe = sq.QueryRecipe()
        recipe["composite"] = sq.collection(red_band, green_band, blue_band).\
            concatenate("band").\
            filter_time("before", sq.time_instant("2019-12-31")).\
            assign(-99)
        result = {
            'reflectance_s2_band02': 525936,
            'reflectance_s2_band03': 525936,
            'reflectance_s2_band04': 525936,
        }
        self.tests.append({
            "name": "no.6b",
            "desc": "temporal filter followed by assign operation",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 6c
        recipe = sq.QueryRecipe()
        recipe["composite"] = sq.collection(red_band, green_band, blue_band).\
            concatenate("band").\
            filter_time("before", sq.time_instant("2019-12-31")).\
            assign_time("month")
        result = {
            'reflectance_s2_band02': 525936,
            'reflectance_s2_band03': 525936,
            'reflectance_s2_band04': 525936,
        }
        self.tests.append({
            "name": "no.6c",
            "desc": "temporal filter followed by assign operation",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })


        # define test no. 7a
        recipe = sq.QueryRecipe()
        recipe["objects"] = sq.reflectance("s2_band04").\
                delineate().\
                filter_time("year", "less", 2020).\
                reduce("mean", "time")
        result = {
            'reflectance_s2_band04': 525960,
        }
        self.tests.append({
            "name": "no.7a",
            "desc": "temporal filter before reduce",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 7b
        recipe = sq.QueryRecipe()
        recipe["objects"] = sq.reflectance("s2_band04").\
                delineate().\
                filter_time("year", "less", 2020).\
                reduce("mean")
        result = {
            'reflectance_s2_band04': 525960,
        }
        self.tests.append({
            "name": "no.7b",
            "desc": "temporal filter before reduce",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 7c
        recipe = sq.QueryRecipe()
        recipe["objects"] = sq.reflectance("s2_band04").\
                delineate().\
                reduce("mean", "space").\
                filter_time("year", "less", 2020)
        result = {
            'reflectance_s2_band04': 525960,
        }
        self.tests.append({
            "name": "no.7c",
            "desc": "temporal filter after reduce-over-space",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 7d
        recipe = sq.QueryRecipe()
        recipe["vegetation_count_per_season"] = sq.reflectance("s2_band04").\
            filter_time("year", "less", 2020).\
            groupby(sq.collection(
                sq.self().\
                    filter_time("year", "less", 2020).\
                    extract("time", "month"), 
                sq.self().\
                    extract("time", "month").\
                    evaluate("not_in", sq.interval(6, 11))
            ).compose()).\
            reduce("count", "time")
        result = {
            'reflectance_s2_band04': 525960,
        }
        self.tests.append({
            "name": "no.7d",
            "desc": "temporal filter with reduce on collection",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })
 
        # define test no. 8a
        recipe = sq.QueryRecipe()
        recipe["objects"] = sq.reflectance("s2_band04").delineate().filter_time("year", "less", 2020)
        recipe["map"] = sq.collection(blue_band, green_band, red_band).\
                concatenate("band").\
                groupby(sq.result("objects"))
        result = {
            'reflectance_s2_band02': 525960,
            'reflectance_s2_band03': 525960,
            'reflectance_s2_band04': 525960,
        }
        self.tests.append({
            "name": "no.8a",
            "desc": "temporal filter indirectly via groupby",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 8b
        recipe = sq.QueryRecipe()
        recipe["objects"] = sq.reflectance("s2_band04").\
                delineate().\
                filter_time("year", "less", 2020).\
                reduce("mean", "time")
        recipe["map"] = sq.collection(blue_band, green_band, red_band).\
                concatenate("band").\
                groupby(sq.result("objects"))
        result = {
            'reflectance_s2_band02': self.max_items,
            'reflectance_s2_band03': self.max_items,
            'reflectance_s2_band04': self.max_items,
        }
        self.tests.append({
            "name": "no.8b",
            "desc": "temporal filter via groupby inactive due to reduce",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # # note: the same tests as 8a/8b could be written for filter_time
        # # however due to issue #55, the test will currently result in unexpected results
        # # src:
        # recipe["res1"] = sq.reflectance("s2_band03").\
        #         delineate().\
        #         filter_time("year", "less", 2020)
        #         # reduce("mean", "time")
        # recipe["res2"] = sq.reflectance("s2_band03").\
        #         filter(sq.result("res1"))

        # define test no. 9a
        recipe = sq.QueryRecipe()
        recipe["res1"] = sq.reflectance("s2_band03").\
                filter_time("year", "less", 2020).\
                reduce("mean", "time")
        recipe["res2"] = sq.reflectance("s2_band03").\
                filter_time("year", "greater_equal", 2020)
        recipe["result"] = sq.collection(sq.result("res1"), sq.result("res2")).\
                concatenate("band")
        result = {
            'reflectance_s2_band03': self.max_items,
        }
        self.tests.append({
            "name": "no.9a",
            "desc": "temporally filtered results in concatenation",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 9b
        recipe = sq.QueryRecipe()
        recipe["res1"] = sq.reflectance("s2_band03").\
                filter_time("year", "less", 2020).\
                reduce("mean", "time")
        recipe["res2"] = sq.reflectance("s2_band04").\
                filter_time("year", "greater_equal", 2020).\
                filter_time("year", "less_equal", 2020)
        recipe["result"] = sq.collection(sq.result("res1"), sq.result("res2")).\
                concatenate("band")
        result = {
            'reflectance_s2_band03': 525960,
            'reflectance_s2_band04': 8784
        }
        self.tests.append({
            "name": "no.9b",
            "desc": "temporally filtered results in concatenation",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })
       
        # define test no. 10a
        recipe = sq.QueryRecipe()
        recipe["composite"] = sq.collection(red_band, green_band, blue_band).\
            concatenate("band").\
            filter_time("year", "less", 2020).\
            groupby_space("feature").\
            reduce("count", "space")
        result = {
            'reflectance_s2_band02': 525960,
            'reflectance_s2_band03': 525960,
            'reflectance_s2_band04': 525960,
        }
        self.tests.append({
            "name": "no.10a",
            "desc": "temporally filtered results with spatial ops",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 10b
        recipe = sq.QueryRecipe()
        recipe["composite"] = sq.collection(red_band, green_band, blue_band).\
            concatenate("band").\
            filter_space("feature", "equal", 0).\
            filter_time("year", "less", 2020)
        result = {
            'reflectance_s2_band02': 525960,
            'reflectance_s2_band03': 525960,
            'reflectance_s2_band04': 525960,
        }
        self.tests.append({
            "name": "no.10b",
            "desc": "temporally filtered results with spatial ops",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 11
        def make_true(obj, track_types = True, **kwargs):
            newobj = obj.copy(deep = True)
            newobj.values = np.ones_like(newobj)
            if track_types:
                newobj.sq.value_type = "binary"
                del obj.sq.value_labels
            return newobj
        def modulus(x, y, track_types = True, **kwargs):
            if track_types:
                manual = {"continuous": {"continuous": "continuous"}, "__preserve_labels": 0}
                promoter = TypePromoter(x, y, manual = manual)
                promoter.check()
            f = lambda x, y: np.mod(x, y)
            y = xr.DataArray(y).sq.align_with(x)
            out = xr.apply_ufunc(f, x, y)
            if track_types:
                out = promoter.promote(out)
            return out
        def sum_of_squares(x, track_types = False, **kwargs):
            if track_types:
                promoter = TypePromoter(x, function = "sum")
                promoter.check()
            f = lambda x, axis = None: np.sum(np.square(x), axis)
            out = x.reduce(f, **kwargs)
            if track_types:
                promoter.promote(out)
            return out
        new_context = copy.deepcopy(self.context)
        new_context["custom_verbs"] = {"make_true": make_true}
        new_context["custom_operators"] = {"modulus": modulus}
        new_context["custom_reducers"] = {"sum_of_squares": sum_of_squares}
        recipe = sq.QueryRecipe()
        recipe["vals"] = sq.reflectance("s2_band04").filter_time("year", "less", 2020)
        recipe["ones"] = sq.result("vals").apply_custom("make_true")
        recipe["mod"] = sq.result("vals").evaluate("modulus", 2)
        recipe["foo"] = sq.result("vals").reduce("sum_of_squares", "space")
        result = {
            'reflectance_s2_band04': 525960,
        }
        self.tests.append({
            "name": "no.11",
            "desc": "temporal filter with custom verbs/ops/reducers",
            "recipe": recipe,
            "result": result,
            "context": new_context
        })

        # define test no. 99a
        recipe = sq.QueryRecipe()
        recipe["res1"] = sq.reflectance("s2_band04").\
                filter_time("year", "less", 2020).\
                evaluate("subtract", sq.reflectance("s2_band03").reduce("mean", "time"))
        recipe["res2"] = sq.reflectance("s2_band08").\
                filter_time("year", "less", 2020).\
                shift("time", 1)
        recipe["res3"] = sq.reflectance("s2_band02").\
                evaluate("less", sq.result("res1"))
        recipe["result"] = sq.collection(sq.result("res1"), sq.result("res2"), sq.result("res3")).\
                concatenate("band").\
                filter_time("year", "less", 2019).\
                reduce("mean", "band")
        result = {
            'reflectance_s2_band02': self.max_items,
            'reflectance_s2_band03': self.max_items,
            'reflectance_s2_band04': 525960,
            'reflectance_s2_band08': 525960
        }
        self.tests.append({
            "name": "no.99a",
            "desc": "complex series of filter & concatenate operations",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        
        # define test no. 99b
        recipe = sq.QueryRecipe()
        recipe["result"] = sq.collection(
                (sq.reflectance("s2_band04").\
                    filter_time("year", "less", 2020).\
                    evaluate("subtract", sq.reflectance("s2_band03").reduce("mean", "time"))),
                (sq.reflectance("s2_band08").\
                    filter_time("year", "greater", 2020)).\
                    filter_time("year", "less", 2023).\
                    shift("time", 1)).\
                concatenate("band").\
                filter_time("year", "less", 2020).\
                reduce("mean", "band")
        result = {
            'reflectance_s2_band03': self.max_items,
            'reflectance_s2_band04': 525960,
            'reflectance_s2_band08': 17520
        }
        self.tests.append({
            "name": "no.99b",
            "desc": "complex series of filter & concatenate operations",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 99c
        recipe = sq.QueryRecipe()
        recipe["result"] = sq.collection(
                (sq.reflectance("s2_band04").\
                    filter_time("year", "less", 2020).\
                    evaluate("subtract", sq.reflectance("s2_band03").reduce("mean", "time"))),
                (sq.reflectance("s2_band08").\
                    filter_time("year", "greater", 2020))).\
                concatenate("band").\
                filter_time("year", "less", 2019).\
                reduce("mean", "band")
        result = {
            'reflectance_s2_band03': self.max_items,
            'reflectance_s2_band04': 517200,
            'reflectance_s2_band08': 0
        }
        self.tests.append({
            "name": "no.99c",
            "desc": "complex series of filter & concatenate operations",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 99d
        recipe = sq.QueryRecipe()
        recipe["res1"] = sq.reflectance("s2_band04").\
                evaluate("subtract", 
                    sq.reflectance("s2_band03").\
                    filter_time("year", "less", 2020).\
                    reduce("mean", "time")
                )
        recipe["res2"] = sq.reflectance("s2_band03").\
                filter_time("year", "less", 2020).\
                reduce("mean", "time").\
                evaluate("subtract", sq.reflectance("s2_band04").reduce("mean", "time"))
        result = {
            'reflectance_s2_band03': 525960, 
            'reflectance_s2_band04': self.max_items,
        }
        self.tests.append({
            "name": "no.99d",
            "desc": "complex series of filter & evaluate operations",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })

        # define test no. 99e
        recipe = sq.QueryRecipe()
        recipe["res"] = sq.collection(
                    sq.collection(
                        blue_band.filter(sq.self().extract("time").evaluate("during", sq.time_interval("2021-01-01", "2021-12-31"))),
                        green_band 
                    ).\
                    compose().\
                    filter_time("year", "greater", 2010),
                    red_band 
                ).\
                merge("any").\
                filter(sq.self().extract("time", "year").evaluate("less", 2015))
        result = {
            'reflectance_s2_band02': 0,
            'reflectance_s2_band03': 35064,
            'reflectance_s2_band04': 482136
        }
        self.tests.append({
            "name": "no.99e",
            "desc": "complex series of filter & compose/merge operations",
            "recipe": recipe,
            "result": result,
            "context": self.context
        })


    def execute(self):
        for test in self.tests:
            print(f"Running test : {test['name']}")
            # run recipe with QueryProcessor
            try:
                if "meta_timestamps" in test["context"]:
                    qp_context = copy.deepcopy(test["context"])
                    del qp_context["meta_timestamps"]
                else:
                    qp_context = test["context"]
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    fp = QueryProcessor.parse(test['recipe'], **qp_context)
            except Exception as e:
                print("Recipe not executable with usual QueryProcessor")
                print("Error message: ", e)
            # run recipe with FilterProcessor
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                fp = FilterProcessor.parse(test['recipe'], **test['context'])
                self.max_items = len(fp.meta_timestamps)
                response = fp.optimize().execute()
            # check result
            try:
                res_shape = {k:len(v) for k,v in response.items()}
                for key, value in test["result"].items():
                    if value is None:
                        test["result"][key] = self.max_items
                if "meta_timestamps" in test["context"]:
                    print(f"Resulting shape: {res_shape}")
                else:
                    assert res_shape == test["result"]
            except Exception as e:
                print("Test failed")
                print(f"Expected shape: {test['result']}")
                print(f"Resulting shape: {res_shape}")

In [5]:
# run all tests
tests = TestSuite()
tests.populate()
tests.execute()

Running test : no.1
Running test : no.2a
Running test : no.2b
Running test : no.3a
Running test : no.3b
Running test : no.3c
Running test : no.3d
Running test : no.3e
Running test : no.3f
Running test : no.4a
Running test : no.4b
Running test : no.4c
Running test : no.5a
Running test : no.5b
Running test : no.5c
Running test : no.5d
Running test : no.6a
Running test : no.6b
Running test : no.6c
Running test : no.7a
Running test : no.7b
Running test : no.7c
Running test : no.7d
Running test : no.8a
Running test : no.8b
Running test : no.9a
Running test : no.9b
Running test : no.10a
Running test : no.10b
Running test : no.11
Running test : no.99a
Running test : no.99b
Running test : no.99c
Running test : no.99d
Running test : no.99e


In [6]:
# To-Do
# inclusion of FilterProcessor in the main branch with connection to datacubes 
# https://datacube-core.readthedocs.io/en/latest/api/indexed-data/generate/datacube.Datacube.list_measurements.html#datacube.Datacube.list_measurements

# Wednesday & Thursday: refactor/adjust reauth (resign function)
# Friday: paper writing & re-run of application examples

In [7]:
# to create a new test or assess an individual one
recipe = sq.QueryRecipe()

tests = TestSuite()
qp_context = copy.deepcopy(tests.context)
if "meta_timestamps" in qp_context:
    del qp_context["meta_timestamps"]
    qp_context["track_types"] = True

red_band = sq.reflectance("s2_band04")
green_band = sq.reflectance("s2_band03")
blue_band = sq.reflectance("s2_band02")

recipe["res1"] = sq.reflectance("s2_band03").\
        evaluate("in", sq.label("test")).\
        filter_time("year", "less", 2020)

# to try new filter processor
# fp = QueryProcessor.parse(recipe, **qp_context)
fp = FilterProcessor.parse(recipe, **qp_context)
response = fp.optimize().execute()
# response

# np.isnan(response["composite"]).sum(('x', 'y'))
# np.all(np.isnan(response["composite"].isel(time=1)))
{k:len(v) for k,v in response.items()}

# np.isnan(response["composite"]).sum(('x', 'y'))
# np.all(np.isnan(response["composite"].isel(time=1)))

{'reflectance_s2_band03': 525960}